In [57]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import glob
import os
import csv

def load_tensile_csv(csv_path):
    """
    Loads tensile raw data CSVs that may contain:
      - metadata header lines
      - mixed delimiters
      - inconsistent column counts
    Returns a cleaned DataFrame with numeric columns only.
    """

    # First: Detect delimiter
    with open(csv_path, "r") as f:
        sample = f.read(2048)

    try:
        sniffer = csv.Sniffer()
        dialect = sniffer.sniff(sample, delimiters=[',',';','\t',' '])
        delimiter = dialect.delimiter
    except:
        delimiter = ','  # fallback

    # Second: Read the file while skipping bad rows
    df = pd.read_csv(
        csv_path,
        sep=delimiter,
        engine="python",
        comment='#',
        skiprows=5,
        #on_bad_lines='skip',    # <-- prevents crash
        skip_blank_lines=True
    )

    # Third: Drop EMPTY or non-numeric columns
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        except:
            df[col] = pd.NA

    df = df.dropna(axis=1, how='all')  # remove columns with no data
    df = df.dropna(axis=0, how='any')  # remove unfinished rows

    print(df)
    return df

def compute_youngs_modulus(
    csv_path,
    gauge_length,
    cross_section_area,
    load_col='Load',
    disp_col='Extension',
    elastic_strain_max=0.005
):
    """
    Computes Young's modulus from tensile test data.

    Parameters
    ----------
    csv_path : str
        Path to CSV file containing load/displacement data.
    gauge_length : float
        Original specimen length (m).
    cross_section_area : float
        Cross-sectional area (m^2).
    load_col : str
        Column name for load/force.
    disp_col : str
        Column name for displacement/extension.
    elastic_strain_max : float
        Maximum strain to use for the linear elastic fit (default = 0.5%).

    Returns
    -------
    E : float
        Estimated Young’s modulus (Pa).
    stress : ndarray
    strain : ndarray
        Arrays of engineering stress and strain.
    """

    # Load file
    df = load_tensile_csv(csv_path)

    # Extract data
    force = df[load_col].values
    disp  = df[disp_col].values

    # Engineering strain
    strain = disp / gauge_length

    # Engineering stress
    stress = force / cross_section_area

    # Select elastic region
    mask = strain <= elastic_strain_max
    strain_elastic = strain[mask].reshape(-1, 1)
    stress_elastic = stress[mask]

    # Linear regression (slope = E)
    model = LinearRegression().fit(strain_elastic, stress_elastic)
    E = model.coef_[0]

    return E, stress, strain


In [58]:
def ultimate_tensile_strength(stress):
    """
    Maximum stress achieved in the test.
    """
    return np.max(stress)

def offset_yield_strength(E, strain, stress, offset=0.02):
    """
    Computes the offset yield strength (default 2%).
    Uses interpolation to find intersection of stress-strain curve with
    offset line of slope equal to the elastic modulus.
    """
    offset_line = E*(strain - offset)

    # difference between actual curve and offset line
    diff = stress - offset_line
    
    # find zero crossing
    idx = np.where(np.diff(np.sign(diff)))[0]
    if len(idx) == 0:
        return None  # No yield point found
    
    i = idx[0]
    # linear interpolation for better accuracy
    x1, x2 = strain[i], strain[i+1]
    y1, y2 = diff[i], diff[i+1]
    
    strain_yield = x1 - y1*(x2 - x1)/(y2 - y1)
    stress_yield = np.interp(strain_yield, strain, stress)
    
    return stress_yield

def total_energy(strain, stress):
    """
    Integrates the stress-strain curve using the trapezoidal rule.
    Units depend on stress/strain units (e.g., MPa -> MJ/m^3).
    """
    return np.trapz(stress, strain)

def strain_at_fracture(strain):
    """
    Maximum strain recorded before failure.
    """
    return np.max(strain)

In [78]:
def analyze_tensile_folder(folder_path, gauge_length, cross_section_area):
    """
    Processes all CSV tensile data files in a folder and returns a summary table.
    """

    results = []

    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    print(csv_files)

    for csv_path in csv_files:
        try:
            # --- Stress & strain + Young's modulus ---
            E, stress, strain = compute_youngs_modulus(
                csv_path,
                gauge_length,
                cross_section_area
            )
            print("fail 1")

            # --- Individual metrics ---
            uts = ultimate_tensile_strength(stress)
            print("fail1.5")
            #ys  = offset_yield_strength(strain, stress, E)
            print("fail1.5")
            energy = total_energy(strain, stress)
            print("fail1.5")

            eps_f = strain_at_fracture(strain)
            print("fail 2")
            results.append({
                "Filename": os.path.basename(csv_path),
                "Youngs_Modulus_Pa": E,
                "UTS_Pa": uts,
            #    "YieldStrength_2pct_Pa": ys,
                "TotalEnergy_Jperm3": energy,
                "StrainAtFracture": eps_f
            })

            print(results, uts, energy, eps_f)

        except Exception as e:
            print(f"Error processing {csv_path}: {e}")

    return pd.DataFrame(results)


In [ ]:
reference_tests = analyze_tensile_folder(
     "./Reference Tests/Reference_T1",
    1, 1
)

print(reference_tests)


['./Reference Tests/Reference_T1\\Specimen_RawData_10.csv', './Reference Tests/Reference_T1\\Specimen_RawData_2.csv', './Reference Tests/Reference_T1\\Specimen_RawData_3.csv', './Reference Tests/Reference_T1\\Specimen_RawData_4.csv', './Reference Tests/Reference_T1\\Specimen_RawData_5.csv', './Reference Tests/Reference_T1\\Specimen_RawData_6.csv', './Reference Tests/Reference_T1\\Specimen_RawData_7.csv', './Reference Tests/Reference_T1\\Specimen_RawData_8.csv', './Reference Tests/Reference_T1\\Specimen_RawData_9.csv']
       Time  Extension         Load
1     0.000    0.00000      7.53494
2     0.100    0.00456      7.72691
3     0.200    0.01371     29.62387
4     0.300    0.02301    122.58210
5     0.400    0.03210    211.75502
..      ...        ...          ...
761  75.428    6.28584  10707.66406
762  75.430    6.28605   9944.93262
763  75.432    6.28627   9164.92188
764  75.434    6.28647   8384.79590
765  75.436    6.28667   7611.41309

[765 rows x 3 columns]
fail 1
fail1.5
fail1